In [4]:
from rdkit import Chem

with Chem.SDMolSupplier('XX01ZVNS2B.corrected.sdf') as sdfh:
    hitdex = {h.GetProp('_Name'): h for h in sdfh}

In [7]:
from typing import List, Dict
from rdkit import Chem

def get_chemical_isomorphisms(mol: Chem.Mol, suffix='-iso') -> List[Chem.Mol]:
    """
    Given a molecule, return a list of molecules where crystallographical ambiguity is resolved.
    """
    query: Chem.Mol = create_unelemental_query(mol)
    # these include all isomorphisms
    to_idx_map = lambda indxs: dict(zip(range(mol.GetNumHeavyAtoms()), indxs))  # noqa
    idx_maps = [to_idx_map(qidxs) for qidxs in mol.GetSubstructMatches(query, uniquify=False)]
    # these are replacement isomorphism
    idx_maps = remove_elemental_isomorphisms(idx_maps, mol)
    if len(idx_maps) == 0:
        raise ValueError(f'An unexplained error: no replacement_isomorphic mappings - before filtering: {idx_maps}')
    elif len(idx_maps) == 1:
        return [mol]
    else:
        return maps_to_mols(idx_maps, mol, suffix)


# dependent functions

def maps_to_mols(idx_maps: List[Dict[int, int]], mol: Chem.Mol, suffix) -> List[Chem.Mol]:
    conformer: Chem.Conformer = mol.GetConformer()  # noqa
    copies = []
    name = mol.GetProp('_Name') if mol.HasProp('_Name') else '_'
    for mi, mapping in enumerate(idx_maps):
        if mi == 0:
            # original was spiked to the front of the mappings
            copies.append(mol)
            continue
        copy = Chem.Mol(mol)
        coconfomer = copy.GetConformer()  # noqa
        for q, t in mapping.items():
            coconfomer.SetAtomPosition(q, conformer.GetAtomPosition(t))
        copy.SetProp('_Name', f'{name}{suffix}{mi}')
        copies.append(copy)
    return copies


def remove_elemental_isomorphisms(idx_maps: List[Dict[int, int]], mol: Chem.Mol) -> List[Dict[int, int]]:
    """
    Benzene can be mapped twelve ways. I do not want this. Only maps where the elements differ
    """
    unique = []
    seen = []
    # spiking in the original for zeroth position
    for m in [dict(zip(range(mol.GetNumHeavyAtoms(), mol.GetNumHeavyAtoms())))] + idx_maps:  # noqa
        # this could be done mathematically, but this is quicker to write
        element_hash = ''.join([mol.GetAtomWithIdx(i).GetSymbol() for i in m.values()])
        if element_hash in seen:
            continue
        unique.append(m)
        seen.append(element_hash)
    return unique


def create_unelemental_query(mol: Chem.Mol) -> Chem.Mol:
    """
    A Query mol where the elements are stripped.
    Changing the atomic Zahlen to 0, will not work as the atoms have to be QueryAtoms,
    which is what `arthorian_quest.enquire` does.
    """
    from arthorian_quest import enquire

    subs = {}
    for atom in mol.GetAtoms():   # noqa
        if atom.GetSymbol() == 'H':
            subs[atom.GetIdx()] = None
        elif atom.GetSymbol() not in 'CNO':
            continue
        elif not atom.GetIsAromatic():
            subs[atom.GetIdx()] = '[C,N,O]'
        else:
            subs[atom.GetIdx()] = 'a'
    return enquire(mol, subs)

In [28]:
isos = get_chemical_isomorphisms(hitdex['x0559_0B'])[1:]

In [29]:
from fragmenstein import Laboratory
from pathlib import Path
import pandas as pd

df1 = pd.DataFrame([{'hits': [iso], 'name': iso.GetProp('_Name').split('-')[0]+f'-iso{i}p', 'smiles': 'C=Cc1c[nH]cn1'} for i, iso in enumerate(isos)])
df2 = pd.DataFrame([{'hits': [iso], 'name': iso.GetProp('_Name').split('-')[0]+f'-iso{i}o', 'smiles': 'C=Cc1cnc[nH]1'} for i, iso in enumerate(isos)])
df = pd.concat([df1, df2]).reset_index(drop=True)

lab = Laboratory(Path('template2.pdb').read_text())
df = lab.place(df)

,smiles,name,binary_hits,error,mode,∆∆G,∆G_bound,∆G_unbound,comRMSD,N_constrained_atoms,...,LE,unminimized_mol,minimized_mol,hit_mols,hit_names,percent_hybrid,largest_ring,N_HA,N_rotatable_bonds,outcome
2,C=Cc1cnc[nH]1,x0559-0B-iso0o,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0f\x00\x0...,,expansion,-2.389884,-2.363922,0.025963,0.368509,7,...,0.341412,<rdkit.Chem.rdchem.Mol object at 0x169ce76a0>,<rdkit.Chem.rdchem.Mol object at 0x169ce6ca0>,[<rdkit.Chem.rdchem.Mol object at 0x169ce4220>],[x0559_0B-iso1],0.0,5,7,1,acceptable
1,C=Cc1c[nH]cn1,x0559-0B-iso1p,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0f\x00\x0...,,expansion,-2.170384,-2.144569,0.025814,0.358237,7,...,0.310055,<rdkit.Chem.rdchem.Mol object at 0x169ce7470>,<rdkit.Chem.rdchem.Mol object at 0x169ce78d0>,[<rdkit.Chem.rdchem.Mol object at 0x169d33b50>],[x0559_0B-iso2],0.0,5,7,1,acceptable
0,C=Cc1c[nH]cn1,x0559-0B-iso0p,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0f\x00\x0...,,expansion,-2.160574,-2.134760,0.025814,0.279411,7,...,0.308653,<rdkit.Chem.rdchem.Mol object at 0x169ce7740>,<rdkit.Chem.rdchem.Mol object at 0x169ce7420>,[<rdkit.Chem.rdchem.Mol object at 0x169ce58f0>],[x0559_0B-iso1],0.0,5,7,1,acceptable
3,C=Cc1cnc[nH]1,x0559-0B-iso1o,[b'\xef\xbe\xad\xde\x00\x00\x00\x00\x0f\x00\x0...,,expansion,-2.067855,-2.041892,0.025963,0.312871,7,...,0.295408,<rdkit.Chem.rdchem.Mol object at 0x169ce74c0>,<rdkit.Chem.rdchem.Mol object at 0x169ce58a0>,[<rdkit.Chem.rdchem.Mol object at 0x169ce4810>],[x0559_0B-iso2],0.0,5,7,1,acceptable


In [32]:
 from rdkit import Chem

with Chem.SDMolSupplier('XX01ZVNS2B.corrected.sdf') as sdfh:
    hits = list(sdfh)

In [41]:
import numpy as np
from rdkit.Chem import AllChem, Descriptors
    
np.median(list(map(Descriptors.ExactMolWt, hits))),\
np.median(list(map(Chem.Mol.GetNumHeavyAtoms, hits)))

(175.60295521, 12.5)

In [ ]:
color coral, resn LIG and element C
color turquoise, polymer and element C
zoom resn LIG
show sticks

set grid_mode, 1
set_name sele_polar_conts, polar_iso0o
set grid_slot, 1, polar_iso0o x0559-0B-iso0o.holo_minimised
set_name sele_polar_conts, polar_iso0p
set grid_slot, 2, polar_iso0p x0559-0B-iso0p.holo_minimised
set_name sele_polar_conts, polar_iso1o
set grid_slot, 3, polar_iso1o x0559-0B-iso1o.holo_minimised
set_name sele_polar_conts, polar_iso1p
set grid_slot, 4, polar_iso1p x0559-0B-iso1p.holo_minimised